# 공유 레이어

함수형 API의 또 다른 유용한 사용처는 공유 레이어를 사용하는 모델입니다. 공유 레이어에 대해서 알아봅시다.

트윗 데이터셋을 고려해 봅시다. 두 개의 다른 트윗을 두고 동일한 사람이 작성했는지 여부를 가려내는 모델을 만들고자 합니다 (이는, 예를 들어 트윗의 유사성을 기준으로 사용자를 비교할 수 있도록 합니다).

이를 구현하는 방법의 하나는 두 개의 트윗을 두 벡터로 인코딩하고 그 두 벡터를 연결한 후 로지스틱 회귀를 더하는 모델을 만드는 것입니다; 이는 두 트윗이 동일한 저자를 공유할 확률을 출력합니다. 그에 따라 모델은 양성 트윗 쌍과 음성 트윗 쌍에 대해서 학습됩니다.

문제가 대칭적이므로, 첫 번째 트윗을 인코딩하는 메커니즘을 (가중치 등을 포함해) 전부 재사용하여 두 번째 트윗을 인코딩해야 합니다. 이 예시에서는 공유된 장단기 메모리 레이어를 사용해 트윗을 인코딩 합니다.

함수형 API로 이 모델을 만들어 봅시다. (280, 256) 형태의 이진 행렬, 다시 말해 256 크기의 벡터 280개로 이루어진 시퀀스를 트윗에 대한 인풋으로 받습니다 (여기서 256 차원 벡터의 각 차원은 가장 빈번한 256 부호의 자모 중 해당 부호의 유/무를 인코딩합니다).

import keras


In [1]:
import keras
from keras.layers import Input, LSTM, Dense
from keras.models import Model

Using TensorFlow backend.


In [2]:
tweet_a = Input(shape=(280, 256))
tweet_b = Input(shape=(280, 256))

In [3]:
# 이 레이어는 행렬을 인풋으로 전달받고
# 64 크기의 벡터를 반환합니다
shared_lstm = LSTM(64)

In [4]:
# 동일한 레이어 인스턴스를
# 여러 번 재사용하는 경우, 레이어의
# 가중치 또한 재사용됩니다
# (그렇기에 이는 실질적으로 *동일한* 레이어입니다)
encoded_a = shared_lstm(tweet_a)
encoded_b = shared_lstm(tweet_b)

In [5]:
# 이제 두 벡터를 연결할 수 있습니다:
merged_vector = keras.layers.concatenate([encoded_a, encoded_b], axis=-1)

In [6]:
# 그리고 로지스틱 회귀를 상층에 추가합니다
predctions = Dense(1, activation='sigmoid')(merged_vector)

In [7]:
# 트윗 인풋을 예측에 연결하는
# 학습 가능한 모델을 정의합니다
model = Model(inputs=[tweet_a, tweet_b], outputs=predctions)

In [8]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy',
             metrics=['accuracy'])

In [ ]:
model.fit([data_a, data_b], labels, epochs=10)

In [10]:
a = Input(shape=(280, 256))
lstm = LSTM(32)
encoded_a = lstm(a)


In [12]:
lstm.output

<tf.Tensor 'lstm_3/strided_slice_18:0' shape=(None, 32) dtype=float32>

In [13]:
encoded_a

<tf.Tensor 'lstm_3/strided_slice_18:0' shape=(None, 32) dtype=float32>

In [16]:
b = Input(shape=(280,256))
encoded_b = lstm(b)

In [17]:
lstm.output

AttributeError: Layer lstm_3 has multiple inbound nodes, hence the notion of "layer output" is ill-defined. Use `get_output_at(node_index)` instead.

In [18]:
lstm.get_output_at(0)

<tf.Tensor 'lstm_3/strided_slice_18:0' shape=(None, 32) dtype=float32>

In [19]:
lstm.get_output_at(1)

<tf.Tensor 'lstm_3_1/strided_slice_6:0' shape=(None, 32) dtype=float32>

In [25]:
from keras.layers import Conv2D

c = Input(shape=(32, 32, 3))
d = Input(shape=(64, 64, 3))

conv = Conv2D(16, (3, 3), padding='same')
conved_c = conv(c)
conved_d = conv(d)

In [26]:
conv.get_input_shape_at(0)

(None, 32, 32, 3)

In [27]:
conv.get_input_shape_at(1)

(None, 64, 64, 3)

In [28]:
conv.get_output_shape_at(0)

(None, 32, 32, 16)

In [29]:
conv.get_output_shape_at(1)

(None, 64, 64, 16)

# inception 모듈

In [32]:
from keras.layers import Conv2D, MaxPooling2D, Input

input_img = Input(shape=(256, 256, 3))

tower_1 = Conv2D(64, (1, 1), padding='same', activation='relu')(input_img)
tower_1 = Conv2D(64, (3, 3), padding='same', activation='relu')(tower_1)

tower_2 = Conv2D(64, (1, 1), padding='same', activation='relu')(input_img)
tower_2 = Conv2D(64, (5, 5), padding='same', activation='relu')(tower_2)

tower_3 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(input_img)
tower_3 = Conv2D(64, (1, 1), padding='same', activation='relu')(tower_3)

output = keras.layers.concatenate([tower_1, tower_2, tower_3], axis=1)

In [33]:
output.shape

TensorShape([None, 768, 256, 64])

In [34]:
tower_1

<tf.Tensor 'conv2d_9/Relu:0' shape=(None, 256, 256, 64) dtype=float32>

In [35]:
tower_2

<tf.Tensor 'conv2d_11/Relu:0' shape=(None, 256, 256, 64) dtype=float32>

In [36]:
tower_3

<tf.Tensor 'conv2d_12/Relu:0' shape=(None, 256, 256, 64) dtype=float32>

# 컨볼루션 레이어에 대한 잔여 연결

In [37]:
from keras.layers import Conv2D, Input

In [39]:
x = Input(shape=(256, 256,3))

In [40]:
y = Conv2D(3, (3, 3), padding='same')(x)

In [41]:
z = keras.layers.add([x, y])

In [42]:
x

<tf.Tensor 'input_15:0' shape=(None, 256, 256, 3) dtype=float32>

In [43]:
y

<tf.Tensor 'conv2d_13/BiasAdd:0' shape=(None, 256, 256, 3) dtype=float32>

In [44]:
z

<tf.Tensor 'add_1/add:0' shape=(None, 256, 256, 3) dtype=float32>

# 시각 공유 모델

In [45]:
from keras.layers import Conv2D, MaxPooling2D, Input, Dense, Flatten
from keras.models import Model

digit_input = Input(shape=(27, 27, 1))
x = Conv2D(64, (3, 3))(digit_input)
x = Conv2D(64, (3, 3))(x)
x = MaxPooling2D((2, 2))(x)
out = Flatten()(x)

vision_model = Model(digit_input, out)

In [46]:
digit_a = Input(shape=(27, 27, 1))
digit_b = Input(shape=(27, 27, 1))

out_a = vision_model(digit_a)
out_b = vision_model(digit_b)

concatenated = keras.layers.concatenate([out_a, out_b])
out = Dense(1, activation='sigmoid')(concatenated)

classification_model = Model([digit_a, digit_b], out)

In [47]:
digit_input

<tf.Tensor 'input_16:0' shape=(None, 27, 27, 1) dtype=float32>

In [48]:
x

<tf.Tensor 'max_pooling2d_3/MaxPool:0' shape=(None, 11, 11, 64) dtype=float32>

In [50]:
out

<tf.Tensor 'dense_2/Sigmoid:0' shape=(None, 1) dtype=float32>

In [51]:
vision_model

In [52]:
digit_a

<tf.Tensor 'input_17:0' shape=(None, 27, 27, 1) dtype=float32>

In [53]:
digit_b

<tf.Tensor 'input_18:0' shape=(None, 27, 27, 1) dtype=float32>

In [54]:
out_a

<tf.Tensor 'model_2/flatten_1/Reshape:0' shape=(None, None) dtype=float32>

In [55]:
out_b

<tf.Tensor 'model_2_1/flatten_1/Reshape:0' shape=(None, None) dtype=float32>

In [56]:
concatenated

<tf.Tensor 'concatenate_4/concat:0' shape=(None, None) dtype=float32>

In [57]:
out

<tf.Tensor 'dense_2/Sigmoid:0' shape=(None, 1) dtype=float32>

In [58]:
classification_model